一、PyTorch基础
- 熟悉 PyTorch 概念和模块。在本快速入门指南中了解如何加载数据、构建深度神经网络、训练和保存模型。
- 开始使用 PyTorch
- 大多数机器学习工作流程涉及处理数据、创建模型、优化模型参数以及保存经过训练的模型。
- 本教程向您介绍在 PyTorch 中实现的完整 ML 工作流程，并提供链接以了解有关每个概念的更多信息。
- 我们将使用 FashionMNIST 数据集来训练神经网络，预测输入图像是否属于以下类别之一：T 恤/上衣、裤子、套头衫、连衣裙、外套、凉鞋、衬衫、运动鞋、包或脚踝启动。

## 1、加载数据集

In [1]:
# PyTorch 有两个处理数据的原语：torch.utils.data.DataLoader 和 torch.utils.data.Dataset。
# Dataset 存储样本及其相应的标签，DataLoader 围绕 Dataset 包装一个可迭代对象。
import torch
from torch import nn
from torch.utils.data import DataLoader
# 计算机视觉数据
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [01:25<00:00, 309448.69it/s] 


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 31753.52it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:11<00:00, 370190.02it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 8453.00it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [3]:
# 我们将数据集作为参数传递给 DataLoader。
# 这在我们的数据集上包装了一个可迭代对象，并支持自动批处理、采样、洗牌和多进程数据加载。
# 这里我们定义批量大小为 64，即数据加载器迭代中的每个元素将返回一批 64 个特征和标签。
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 2、创建模型
- 为了在 PyTorch 中定义神经网络，我们创建一个继承自 nn.Module 的类。
- 我们在 __init__ 函数中定义网络层，并在前向函数中指定数据如何通过网络。
- 为了加速神经网络中的操作，我们将其转移到 GPU 或 MPS（如果可用）。

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 3、优化模型参数
- 为了训练模型，我们需要损失函数和优化器。

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
# 在单个训练循环中，模型对训练数据集（批量输入）进行预测，并反向传播预测误差以调整模型的参数。
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
# 我们还根据测试数据集检查模型的性能，以确保它正在学习。
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
# 训练过程会进行多次迭代（epoch）。
# 在每个时期，模型都会学习参数以做出更好的预测。
# 我们打印每个时期模型的准确性和损失；我们希望看到每个 epoch 的准确率都会提高，损失也会减少。
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297072  [   64/60000]
loss: 2.283575  [ 6464/60000]
loss: 2.267705  [12864/60000]
loss: 2.261562  [19264/60000]
loss: 2.252863  [25664/60000]
loss: 2.224608  [32064/60000]
loss: 2.226368  [38464/60000]
loss: 2.197275  [44864/60000]
loss: 2.187764  [51264/60000]
loss: 2.168748  [57664/60000]
Test Error: 
 Accuracy: 49.7%, Avg loss: 2.155741 

Epoch 2
-------------------------------
loss: 2.167019  [   64/60000]
loss: 2.153379  [ 6464/60000]
loss: 2.096991  [12864/60000]
loss: 2.109651  [19264/60000]
loss: 2.077110  [25664/60000]
loss: 2.010962  [32064/60000]
loss: 2.037788  [38464/60000]
loss: 1.963984  [44864/60000]
loss: 1.961153  [51264/60000]
loss: 1.907285  [57664/60000]
Test Error: 
 Accuracy: 55.0%, Avg loss: 1.895162 

Epoch 3
-------------------------------
loss: 1.924061  [   64/60000]
loss: 1.898052  [ 6464/60000]
loss: 1.780825  [12864/60000]
loss: 1.822613  [19264/60000]
loss: 1.731348  [25664/60000]
loss: 1.667297  [32064/600

## 4、保存模型
- 保存模型的常见方法是序列化内部状态字典（包含模型参数）。

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 5、加载模型
- 加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

## 6、使用模型

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [ ]:
# 上面就是使用训练一个属于自己的模型过程，无论多复杂的模型，步骤基本都一样。